<a href="https://colab.research.google.com/github/lacakp/Project-Mask-Detection/blob/main/Face_Detection_Mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face Detection Mask

## Introduction


In [1]:
# library สำหรับใช่ videocapture
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import numpy as np
from PIL import Image
import io

In [2]:
# โหลด haarcascade เพื่อช่วยในการตรวจจับใบหน้า
!wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
# โหลด model เพื่อทำการตรวจจับหน้ากาก
!wget https://raw.githubusercontent.com/lacakp/Project-Mask-Detection/main/mask_detection_model

--2021-09-25 08:25:17--  https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 930127 (908K) [text/plain]
Saving to: ‘haarcascade_frontalface_default.xml’

haarcascade_frontal 100%[===================>] 908.33K  --.-KB/s    in 0.05s   

2021-09-25 08:25:18 (18.1 MB/s) - ‘haarcascade_frontalface_default.xml’ saved [930127/930127]

--2021-09-25 08:25:18--  https://raw.githubusercontent.com/lacakp/Project-Mask-Detection/main/mask_detection_model
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... conn

## Videocapture

In [3]:
## โค้ด videocapture 
# reference 
# - AI บ้าน บ้าน (รศ.ดร.ปริญญา สงวนสัตย์)
# - https://www.youtube.com/watch?v=1VziTgVt4GQ&t=11s
# - https://colab.research.google.com/drive/1v4zM9Gcxt6r5pHGN8HS6CYsLTt1VoZsG

def VideoCapture(): # ฟังก์ชันสำหรับเรียกใช้ VideoCapture()
  js = Javascript('''
    async function create(){
      div = document.createElement('div');
      document.body.appendChild(div);

      video = document.createElement('video');
      video.setAttribute('playsinline', '');

      div.appendChild(video);

      stream = await navigator.mediaDevices.getUserMedia({video: {facingMode: "environment"}});
      video.srcObject = stream;

      await video.play();

      canvas =  document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);

      div_out = document.createElement('div');
      document.body.appendChild(div_out);
      img = document.createElement('img');
      div_out.appendChild(img);
    }

    async function capture(){
        return await new Promise(function(resolve, reject){
            pendingResolve = resolve;
            canvas.getContext('2d').drawImage(video, 0, 0);
            result = canvas.toDataURL('image/jpeg', 0.8);
            pendingResolve(result);
        })
    }

    function showimg(imgb64){
        img.src = "data:image/jpg;base64," + imgb64;
    }

  ''')
  display(js)

def byte2image(byte): # ฟังก์ชันสำหรับแปลงข้อมูล byte เป็นรูปภาพ
  jpeg = b64decode(byte.split(',')[1])
  im = Image.open(io.BytesIO(jpeg))
  return np.array(im)

def image2byte(image): # ฟังก์ชันสำหรับแปลงข้อมูล ภาพ เป็น byte
  image = Image.fromarray(image)
  buffer = io.BytesIO()
  image.save(buffer, 'jpeg')
  buffer.seek(0)
  x = b64encode(buffer.read()).decode('utf-8')
  return x


## Pre-processing

In [4]:
# Pre-processing
import cv2     # OpenCV provides a real-time optimized Computer Vision library
import numpy as np # NumPy เป็นไลบรารี่ที่เพิ่มการรองรับอาร์เรย์และเมทริกซ์ขนาดใหญ่หลายมิติ พร้อมด้วยคอลเลกชั่นฟังก์ชันทางคณิตศาสตร์ระดับสูงจำนวนมาก
from keras.models import load_model # keras.models เพื่อทำการโหลดโมเดลจากไฟล์
import tensorflow as tf             # TensorFlow เป็นไลบรารีซอฟต์แวร์โอเพ่นซอร์สฟรีสำหรับการเรียนรู้ของเครื่องและปัญญาประดิษฐ์
from google.colab.patches import cv2_imshow # ใช้ cv_imshow สำหรับโชว์ภาพบนโคแลป
model = load_model("./mask_detection_model") # ทำการโหลดโมเดล
face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') 
# ใช้ haarcascade (การตรวจจับวัตถุโดยใช้ตัวแยกประเภทการเรียงซ้อนตามฟีเจอร์ของ Haar เป็นวิธีการตรวจจับวัตถุที่มีประสิทธิภาพซึ่งเสนอโดย Paul Viola และ Michael Jones)

## Processing | Post-Processing

In [ ]:
# processing
# กด CTRL + M + L เพื่อโชว์ จำนวนบรรทัดใน ceil (ซึ่งเป็น คีย์ลัดของ colab)
# model shape  shape=(None, 224, 224, 3) (รูปร่างของโมเดล ซึ่งโมเดลถูกเทรนด้วยภาพ ขนาด 224x224)

VideoCapture()  # เรียกใช้ฟังก์ชัน VideoCapture() เพื่อทำการเรียกใช้กล้อง ซึ่งโค้ดส่วนนี้คือ Java Script เพื่อทำให้ใช้งานกล้องใน colab ได้ โดย รศ.ดร.ปริญญา สงวนสัตย์
eval_js('create()') 
while True: # เริ่มต้นการ ตรวจจับใบหน้าและเช็คว่ามีแมสก์หรือไม่
    byte = eval_js('capture()')  # เป็นข้อมูล byte ที่ได้จากการอ่านภาพจากกล้อง
    frame = byte2image(byte) # แปลง ข้อมูล byte เป็น ภาพ ด้วยฟังก์ชัน byte2image()
    frame = cv2.flip(frame,1,1) # สลับเพื่อให้ไม่เหมือนกระจก
    new_img = cv2.resize(frame, (frame.shape[1] // 1, frame.shape[0] // 1)) # resize ขนาดภาพเพื่อให้ง่ายต่อการตรวจจับ

    faces = face_detector.detectMultiScale(new_img) # สำหรับตรวจจับใบหน้า โดย 
    for x, y, w, h in faces: # วนซ้ำพิกัดบนใบหน้า
      face_img = new_img[y:x+h, x:x+w] # ดึงพิกัดใบหน้า
      resized = cv2.resize(face_img, (224, 224)) # ให้ภาพใบหน้า fit กับโมเดล (224,224)
      img_array = tf.keras.preprocessing.image.img_to_array(resized) # แปลงใบหน้าเป็น array
      img_array = tf.expand_dims(img_array, 0) #ขยายมิติภาพฟิตกับโมดล
      predictions = model.predict(img_array) # ทำนายบน ROI (Region of Interest)
      score = tf.nn.softmax(predictions[0]) # ผลลัพธ์
      label = np.argmax(score) # หาค่าสูงสุด

      # Post-Processing
 
      if label == 0: # ถ้าค่าที่ได้ เป็น 0 
        cv2.rectangle(new_img, (x, y), (x+w, y+h), (0, 255, 0), 2)  # วาดสี่เหลี่ยมรอบใบหน้า
        cv2.putText(new_img, "mask", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2) # แสดงข้อความ "mask"
      elif label == 1:
        cv2.rectangle(new_img, (x, y), (x+w, y+h), (0, 255, 0), 2) # วาดสี่เหลี่ยมรอบใบหน้า
        cv2.putText(new_img, "No mask", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2) # แสดงข้อความ "No mask"
      else:
        None
      # pass
      # แสดงผลหลังจากทำนาย
      new_img = cv2.cvtColor(new_img, cv2.COLOR_BGR2RGB) # ทำการแปลงภาพจาก ปริภูมิ BGR เป็น RGB
      cv2_imshow(new_img) # แสดงภาพ
      print(np.argmax(score), 100 * np.max(score)) # แสดงค่าความถูกต้อง

    eval_js('showimg("{}")'.format(image2byte(frame))) # แสดงเฟรมกล้อง